In [1]:
import pandas as pd

In [2]:
airbnb=pd.read_csv("train.csv")

In [ ]:
airbnb.count()

In [3]:
airbnb=airbnb.drop(columns=['neighbourhood','description','id','name','thumbnail_url','zipcode',\
                            'latitude','longitude','city','first_review','last_review'])

In [ ]:
airbnb.describe()

In [ ]:
airbnb.head(10)

In [ ]:
airbnb.amenities[26]

In [4]:
def count_amen(x):
    return x.count(',')+1

airbnb['amenities']=airbnb['amenities'].apply(count_amen)  #amenities는 하나하나 레이블링 하기에 데이터가 복잡해지므로 개수로 바꿈

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
airbnb.plot(y='log_price', kind='box')

In [ ]:
airbnb.hist(column='log_price',bins=20)

In [ ]:
airbnb.dtypes

In [ ]:
airbnb

In [5]:
import numpy as np
import datetime
airbnb['host_since']=datetime.datetime.now().year-pd.DatetimeIndex(airbnb['host_since']).year

In [ ]:
airbnb.count()

In [6]:
from sklearn.impute import SimpleImputer

In [7]:
imputer=SimpleImputer()

In [8]:
#impute missing values

airbnb[['review_scores_rating']]=imputer.fit_transform(airbnb[['review_scores_rating']])
imputer.set_params(strategy='median')
airbnb[['bathrooms']]=imputer.fit_transform(airbnb[['bathrooms']])
airbnb[['bedrooms']]=imputer.fit_transform(airbnb[['bedrooms']])
airbnb[['beds']]=imputer.fit_transform(airbnb[['beds']])
airbnb[['host_since']]=imputer.fit_transform(airbnb[['host_since']])

imputer.set_params(strategy='most_frequent')
airbnb[['host_has_profile_pic']]=imputer.fit_transform(airbnb[['host_has_profile_pic']])
airbnb[['host_identity_verified']]=imputer.fit_transform(airbnb[['host_identity_verified']])

In [ ]:
airbnb[['review_scores_rating']]

In [9]:
#airbnb.host_response_rate 거의 100프로이고 missing value 많으므로 제외
airbnb=airbnb.drop(columns=['host_response_rate'])

In [10]:
from sklearn.preprocessing import OrdinalEncoder

In [11]:
ord_enc=OrdinalEncoder()

In [12]:
ord_enc.fit(airbnb[['host_has_profile_pic','host_identity_verified','instant_bookable','cleaning_fee','cancellation_policy']])

OrdinalEncoder(categories='auto', dtype=<class 'numpy.float64'>)

In [13]:
ord_enc.categories_

[array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array(['f', 't'], dtype=object),
 array([False, True], dtype=object),
 array(['flexible', 'moderate', 'strict', 'super_strict_30',
        'super_strict_60'], dtype=object)]

In [14]:
airbnb[['host_has_profile_pic','host_identity_verified','instant_bookable','cleaning_fee','cancellation_policy']]\
=ord_enc.transform(airbnb[['host_has_profile_pic','host_identity_verified','instant_bookable','cleaning_fee','cancellation_policy']])

In [19]:
airbnb.head(5)

,log_price,room_type,amenities,accommodates,bathrooms,cancellation_policy,cleaning_fee,host_has_profile_pic,host_identity_verified,host_since,instant_bookable,number_of_reviews,review_scores_rating,bedrooms,beds
0,5.010635,Entire home/apt,9,3,1.0,2.0,1.0,1.0,1.0,7.0,0.0,2,100.000000,1.0,1.0
1,5.129899,Entire home/apt,15,7,1.0,2.0,1.0,1.0,0.0,2.0,1.0,6,93.000000,3.0,3.0
2,4.976734,Entire home/apt,19,5,1.0,1.0,1.0,1.0,1.0,3.0,1.0,10,92.000000,1.0,3.0
3,6.620073,Entire home/apt,15,4,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0,94.067365,2.0,2.0
4,4.744932,Entire home/apt,12,2,1.0,1.0,1.0,1.0,1.0,4.0,1.0,4,40.000000,0.0,1.0


In [ ]:
airbnb['property_type'].value_counts()

In [20]:
airbnb['room_type'].value_counts()

Entire home/apt    41310
Private room       30638
Shared room         2163
Name: room_type, dtype: int64

In [ ]:
airbnb['bed_type'].value_counts()

In [17]:
#property type은 너무 쓸데없이 많은 종류가 있고 apartment에 치우쳐 져있으므로 room_type만 사용하는게 낫다고 생각하여 삭제
#bed type은 real bed가 대부분의 데이터를 차지하므로 잘못 학습할 수 있으므로 삭제
airbnb=airbnb.drop(columns=['property_type','bed_type'])

In [24]:
airbnb=pd.concat((airbnb.drop(columns=['room_type']),pd.get_dummies(airbnb[['room_type']])),axis=1)

In [27]:
airbnb.dtypes

log_price                    float64
amenities                      int64
accommodates                   int64
bathrooms                    float64
cancellation_policy          float64
cleaning_fee                 float64
host_has_profile_pic         float64
host_identity_verified       float64
host_since                   float64
instant_bookable             float64
number_of_reviews              int64
review_scores_rating         float64
bedrooms                     float64
beds                         float64
room_type_Entire home/apt      uint8
room_type_Private room         uint8
room_type_Shared room          uint8
dtype: object

In [28]:
airbnb.head(10)

,log_price,amenities,accommodates,bathrooms,cancellation_policy,cleaning_fee,host_has_profile_pic,host_identity_verified,host_since,instant_bookable,number_of_reviews,review_scores_rating,bedrooms,beds,room_type_Entire home/apt,room_type_Private room,room_type_Shared room
0,5.010635,9,3,1.0,2.0,1.0,1.0,1.0,7.0,0.0,2,100.000000,1.0,1.0,1,0,0
1,5.129899,15,7,1.0,2.0,1.0,1.0,0.0,2.0,1.0,6,93.000000,3.0,3.0,1,0,0
2,4.976734,19,5,1.0,1.0,1.0,1.0,1.0,3.0,1.0,10,92.000000,1.0,3.0,1,0,0
3,6.620073,15,4,1.0,0.0,1.0,1.0,1.0,4.0,0.0,0,94.067365,2.0,2.0,1,0,0
4,4.744932,12,2,1.0,1.0,1.0,1.0,1.0,4.0,1.0,4,40.000000,0.0,1.0,1,0,0
5,4.442651,10,2,1.0,2.0,1.0,1.0,1.0,2.0,1.0,3,100.000000,1.0,1.0,0,1,0
6,4.418841,21,3,1.0,1.0,1.0,1.0,0.0,2.0,1.0,15,97.000000,1.0,1.0,1,0,0
7,4.787492,26,2,1.0,1.0,1.0,1.0,1.0,6.0,0.0,9,93.000000,1.0,1.0,1,0,0
8,4.787492,21,2,1.0,1.0,1.0,1.0,0.0,4.0,0.0,159,99.000000,1.0,1.0,0,1,0
9,3.583519,13,2,1.0,1.0,1.0,1.0,0.0,2.0,1.0,2,90.000000,1.0,1.0,0,1,0
